In [2]:
import pandas as pd
from kmers_frequency_encoding import read_fasta_to_kmers

/home/tzutang/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
train_df = pd.read_csv('train.csv')
test_df =  pd.read_csv('test.csv')
# train_df['group'].value_counts(), test_df['group'].value_counts()

In [4]:
train_kmers_df = read_fasta_to_kmers(train_df, file_dir = '../data', k_mers = [3])
test_kmers_df = read_fasta_to_kmers(test_df, file_dir = '../data', k_mers = [3])

#train_kmers_df.to_csv('encoded_data/train_3mers.csv')
#test_kmers_df.to_csv('encoded_data/test_3mers.csv')
# train_3mers_df = pd.read_csv('encoded_data/train_3mers.csv',index_col=[0])
# test_3mers_df = pd.read_csv('encoded_data/test_3mers.csv',index_col=[0])
# train_4mers_df = pd.read_csv('encoded_data/train_4mers.csv',index_col=[0])
# test_4mers_df = pd.read_csv('encoded_data/test_4mers.csv',index_col=[0])

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from joblib import dump
import os

# Initialize models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "XGBoost": GradientBoostingClassifier()
}

# Prepare features and labels
X_train = train_4mers_df.iloc[:, :-2]
y_train = train_4mers_df['group']
X_test = test_4mers_df.iloc[:, :-2]
y_test = test_4mers_df['group']

# List to collect performance data
performance_data = []

# Ensure the model directory exists
model_directory = "WGS_models"
os.makedirs(model_directory, exist_ok=True)

# Train, predict, and save each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    performance_data.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1 Score': f1_score(y_test, y_pred, average='weighted')
    })
    
    # Save the model
    model_path = os.path.join(model_directory, f"{name.replace(' ', '_').lower()}_4mers.joblib")
    dump(model, model_path)

# Create DataFrame from collected performance data
performance_df = pd.DataFrame(performance_data)
print(performance_df)

                  Model  Accuracy  Precision    Recall  F1 Score
0         Random Forest  0.912354   0.911313  0.912354  0.911594
1   Logistic Regression  0.743434   0.761869  0.743434  0.658056
2                   SVM  0.859363   0.855414  0.859363  0.854849
3                   KNN  0.906915   0.905470  0.906915  0.905426
4         Decision Tree  0.873038   0.873196  0.873038  0.873116
5  Gaussian Naive Bayes  0.679409   0.680819  0.679409  0.680104
6               XGBoost  0.845688   0.840809  0.845688  0.838835


In [9]:
performance_df

,Model,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.912354,0.911313,0.912354,0.911594
1,Logistic Regression,0.743434,0.761869,0.743434,0.658056
2,SVM,0.859363,0.855414,0.859363,0.854849
3,KNN,0.906915,0.905470,0.906915,0.905426
4,Decision Tree,0.873038,0.873196,0.873038,0.873116
5,Gaussian Naive Bayes,0.679409,0.680819,0.679409,0.680104
6,XGBoost,0.845688,0.840809,0.845688,0.838835


In [74]:
performance_df

,Model,Accuracy,Precision,Recall,F1 Score
0,decision_tree,0.868376,0.867736,0.868376,0.868036
1,gaussian_naive_bayes,0.675524,0.677122,0.675524,0.676311
2,knn,0.902720,0.901117,0.902720,0.901031
3,logistic_regression,0.749495,0.748497,0.749495,0.679272
4,svm,0.841492,0.836774,0.841492,0.832830
5,random_forest,0.913598,0.912472,0.913598,0.912680
6,xgboost,0.835742,0.829885,0.835742,0.827745


In [49]:
k_mers = [3]
file_dir = '../data_short_read/'
test_df_s = test_df.sample(n=10, random_state=42)
test_kmers_df = read_fasta_to_kmers(test_df_s, file_dir, k_mers)

100%|██████████| 10/10 [00:00<00:00, 127.92it/s]


In [50]:
y_pred = model.predict(test_kmers_df.iloc[:,0:-2])
print(classification_report(test_kmers_df['group'], y_pred))

              precision    recall  f1-score   support

           0       0.48      0.78      0.59    128926
           1       0.70      0.38      0.49    176067

    accuracy                           0.55    304993
   macro avg       0.59      0.58      0.54    304993
weighted avg       0.61      0.55      0.54    304993



In [61]:
test_kmers_df = pd.read('encoded_data/test_3mers_extracted_sr')

In [62]:
y_pred = model.predict(test_kmers_df.iloc[:,0:-2])
print(classification_report(test_kmers_df['group'], y_pred))

              precision    recall  f1-score   support

           0       0.34      0.68      0.45    925000
           1       0.71      0.37      0.48   1978000

    accuracy                           0.47   2903000
   macro avg       0.52      0.52      0.47   2903000
weighted avg       0.59      0.47      0.47   2903000

